<a href="https://colab.research.google.com/github/gustavo-ifusp/prefeitura_smit_sp360/blob/main/saude_dados_remedios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import zipfile
from datetime import date, time, datetime, timedelta
from dateutil.relativedelta import relativedelta

#**Saúde - Dados dos Remédios**
**Dados de origem:**<br>
\\\SMITGBC031\situacao_medicamentos
<br>
**Arquivos backup para teste**<br>
https://drive.google.com/drive/folders/1p1ZnLh8SUBNoygG2m1AwX3IGQ_aWstOP?usp=sharing
<br><br>
**Objetivo:**<br>
Organizar os dados por região e data, deixando no esquema star schema.<br><br>
Responsável: Gustavo de Andrade <br> gustavo.andrade@prefeitura.sp.gov.br


In [ ]:
dados = pd.read_csv("gss_posicao_estoque_20211020.csv", sep = ';', header = 0, encoding='ISO-8859-1')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
dados

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,579
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,506
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,101
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,5
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,500
...,...,...,...,...,...,...,...,...,...,...,...,...
484735,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,40
484736,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,100
484737,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,4
484738,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,1


In [ ]:
dados['TIPO UNIDADE MEDIDA'].unique()

array(['UN ', 'FR ', 'AMP', 'FAM', 'ENV', 'CP ', 'BNG', 'M  ', 'PC ',
       'RL ', 'KIT', 'PCT', 'PAR', 'TST', 'LT ', 'CJ ', 'TB ', 'EST',
       'CAP', 'L  ', 'CX ', 'GL ', 'FL ', 'PT ', 'G  ', 'BOB', 'SRG',
       'ML ', 'DRG', 'M3 ', 'RM ', 'BLC', 'BOL', 'SCH', 'KG ', 'CPD',
       '   ', 'CT ', 'SER'], dtype=object)

In [ ]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484740 entries, 0 to 484739
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SUPERVISAO                483001 non-null  object 
 1   CNES ESTABELECIMENTO      483232 non-null  float64
 2   SUPRI ESTABELECIMENTO     483477 non-null  float64
 3   ESTABELECIMENTO           484740 non-null  object 
 4   CODIGO SUPRI ITEM         484740 non-null  object 
 5   DESCRICAO ITEM            484740 non-null  object 
 6   TIPO UNIDADE MEDIDA       484740 non-null  object 
 7   CODIGO ALMOXARIFADO ITEM  484740 non-null  int64  
 8   STATUS                    484740 non-null  object 
 9   BLOQUEIO STS              484740 non-null  object 
 10  CMM                       484554 non-null  float64
 11  ESTOQUE                   484740 non-null  int64  
dtypes: float64(3), int64(2), object(7)
memory usage: 44.4+ MB


In [ ]:
dados['SUPERVISAO'].unique()

array(['SUPERVISÃO TÉCNICA DE SAÚDE SÉ',
       'SUPERVISAO TECNICA DE SAUDE SANTA CECILIA',
       'SUPERVISÃO TÉCNICA DE SAÚDE ARICANDUVA/FORMOSA/CARRÃO/MOOCA',
       'SUPERVISÃO TÉCNICA DE SAÚDE VILA PRUDENTE/SAPOPEMBA',
       'SUPERVISÃO TÉCNICA DE SAÚDE VILA MARIANA/JABAQUARA',
       'SUPERVISÃO TÉCNICA DE SAÚDE VILA MARIA/VILA GUILHERME',
       'SUPERVISÃO TÉCNICA DE SAÚDE PENHA',
       'SUPERVISÃO TÉCNICA DE SAÚDE ITAQUERA',
       'SUPERVISAO TECNICA DE SAUDE ERMELINO MATARAZZO',
       'SUPERVISÃO TÉCNICA DE SAÚDE ITAIM PAULISTA',
       'SUPERVISÃO TÉCNICA DE SAÚDE IPIRANGA',
       'SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ',
       'SUPERVISAO TECNICA DE SAUDE PERUS', nan,
       'SUPERVISÃO TÉCNICA DE SAÚDE FREGUESIA DO Ó/BRASILÂNDIA',
       'SUPERVISÃO TÉCNICA DE SAÚDE CASA VERDE/CACHOEIRINHA',
       'SUPERVISÃO TÉCNICA DE SAÚDE SANTANA/JAÇANA/TREMEMBE/TUCURUVI',
       'SUPERVISÃO TÉCNICA DE SAÚDE CAPELA DO SOCORRO',
       'SUPERVISÃO TÉCNICA DE SAÚDE MBOI MIRIM',
    

In [ ]:
remedios = pd.DataFrame()
remedios['remedios'] = Counter(dados['DESCRICAO ITEM']).keys()
remedios['repete'] = Counter(dados['DESCRICAO ITEM']).values()
remedios

,remedios,repete
0,PARACETAMOL 500 MG COMPRIMIDO,613
1,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,695
2,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,652
3,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,692
4,DIPIRONA SODICA 500 MG COMPRIMIDO,633
...,...,...
3226,SAMU - DETERGENTE DESINCROSTANTE ORGANICO PARA...,1
3227,SACO DE ALGODAO CRU ALVEJADO - 71 X 41 CM - BR...,1
3228,"DUCHA, HIGIENICA, INTIMA, USO UNICO",1
3229,"CANULA, TRAQUEOSTOMIA, S/ BALAO, DESCARTAVEL, ...",1


#**Começo da modelagem**
Até agora foi para entender o dataframe!

In [ ]:
dados

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,579
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,506
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,101
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,5
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,500
...,...,...,...,...,...,...,...,...,...,...,...,...
484735,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,40
484736,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,100
484737,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,4
484738,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,1


In [ ]:
estoque_regiao = dados.groupby(['SUPERVISAO','DESCRICAO ITEM'])['ESTOQUE'].sum()

In [ ]:
estoque_regiao

SUPERVISAO                              DESCRICAO ITEM                                                                                      
FUNDAÇÃO PARA O REMEDIO POPULAR - FURP  ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAPSULA OU DRAGEAS COM 250 MG                                33400
                                        BENZILPENICILINA PROCAINA 300.000 UI + BENZILPENICILINA POTASSICA 100.000 UI SUSPENSAO INJETAVEL FAM        5
                                        CEFALEXINA 50 MG/ML SUSPENSAO ORAL FRASCO 60 ML                                                           492
                                        CIMETIDINE 200 MG (TAGAMET, ULCIMET)                                                                     2220
                                        DIAZEPAN EM COMPRIMIDOS SULCADOS COM 10 MG                                                               2960
                                                                                                             

In [ ]:
estoque_estabelecimento = dados.groupby(['ESTABELECIMENTO','DESCRICAO ITEM'])['ESTOQUE'].sum()

In [ ]:
estoque_estabelecimento

ESTABELECIMENTO       DESCRICAO ITEM                                                    
AE ALTO DA BOA VISTA  ABAIXADOR, LINGUA, MADEIRA, 100 UNIDADES                                4
                      ACIDO ACETICO 20 MG/ML (2%) SOLUCAO FRASCO 1 LITRO                      1
                      ACIDO ACETICO 50 MG/ML (5%) SOLUCAO FRASCO 1 LITRO                      1
                      ACIDO ACETILSALICILICO 100 MG COMPRIMIDO                               30
                      ACIDO NITRICO FUMEGANTE 660 MG/ML (66%) SOLUCAO FRASCO 10 ML            0
                                                                                           ... 
URSI TUCURUVI         TERBUTALINA 0,5 MG/ML SOLUCAO INJETAVEL AMP. 1 ML                       4
                      TIRA, REAGENTE, P/ DETERMINACAO DE GLICOSE, SANGUE, USO HOSPITALAR      0
                      TOUCA, CIRURGICA, DESCARTAVEL                                           0
                      TOUCA, CIRURGICA, DESCART

In [ ]:
estoque_estabelecimento_cnes = dados.groupby(['CNES ESTABELECIMENTO','DESCRICAO ITEM'])['ESTOQUE'].sum()

In [ ]:
estoque_estabelecimento_cnes

CNES ESTABELECIMENTO  DESCRICAO ITEM                                                                                           
85074.0               AMOXICILINA 50 MG/ML PO PARA SUSPENSAO ORAL FRASCO 150 ML                                                        0
                      AMOXICILINA 500 MG COMPRIMIDO                                                                                24000
                      AZITROMICINA 40 MG/ML SUSPENSAO ORAL FRASCO 15 ML                                                              725
                      AZITROMICINA 500 MG COMPRIMIDO                                                                                6000
                      CEFALEXINA 50 MG/ML SUSPENSAO ORAL FRASCO 60 ML                                                               1000
                                                                                                                                   ...  
9999977.0             TUBO P/ COLETA DE SANGUE A V

#**Adquirindo como teste o backup de arquivos da pasta no Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#**Esse codigo vai extrar todos os arquivos em uma pasta e salvar em outra:**

In [ ]:
arquivos = glob.glob('/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/*.zip') #não precisa mais carregar novamente
arquivos

['/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/gss_posicao_estoque_20211021.zip',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/gss_posicao_estoque_20211022.zip',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/gss_posicao_estoque_20211023.zip',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/gss_posicao_estoque_20211024.zip',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/gss_posicao_estoque_20211025.zip',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/gss_posicao_estoque_20211026.zip',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/gss_posicao_estoque_20211027.zip',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/gss_posicao_estoque_20211028.zip',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/gss_posicao_estoque_20211029.zip',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/gss_posicao_estoque_20211030.zip',


In [ ]:
n = 0
while n < len(arquivos):
    zip = zipfile.ZipFile(arquivos[n])
    zip.extractall('/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraido/')
    n += 1

faltou o arquivo danificado 21/05/2021, farei individualmente

#**Agora o objetivo é ler cada arquivo e criar uma coluna com data que está no nome, depois concatenar todos os arquivos de forma automatica**

In [ ]:
extraidos = glob.glob('/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/abril/*.csv')
extraidos

['/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/abril/gss_posicao_estoque_20210401.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/abril/gss_posicao_estoque_20210402.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/abril/gss_posicao_estoque_20210403.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/abril/gss_posicao_estoque_20210404.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/abril/gss_posicao_estoque_20210405.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/abril/gss_posicao_estoque_20210406.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/abril/gss_posicao_estoque_20210407.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/abril/gss_posicao_estoque_20210408.csv',
 '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraid

In [ ]:
all_data = pd.DataFrame() 
n = 0
for f in extraidos:
    df1 = pd.read_csv(f, sep = ';', header = 0, encoding='ISO-8859-1')
    data_n = extraidos[n].split('/')
    data_n = data_n[9].split('_')
    data_n = data_n[3].split('.')
    data_n = int(data_n[0])
    data_n_ano = data_n // 10000
    data_n_mes = (data_n % 10000)//100
    data_n_dia = data_n % 10000
    data_n_dia = data_n_dia % 100
    df1['data']  = date(year = data_n_ano, month = data_n_mes, day = data_n_dia)
    all_data = all_data.append(df1)
    n += 1
all_data.sample(2)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
189246,SUPERVISÃO TÉCNICA DE SAÚDE VILA PRUDENTE/SAPO...,2774755.0,8.425301e+09,SAE DST/AIDS HERBERT DE SOUZA - BETINHO,1106500300480207,"SERINGA, 10 ML, DESCARTAVEL, ESTERIL",UN,8410200074,DESBLOQUEADO,NAO,40.0,0,2021-04-03
389272,SUPERVISÃO TÉCNICA DE SAÚDE PARELHEIROS,6332447.0,8.424302e+09,UBS JARDIM SANTA FE,1106400904700425,"ESTRIOL 1 MG/G (0,1%) CREME VAGINAL BISNAGA 50 G",BNG,8410200066,DESBLOQUEADO,NAO,10.0,3,2021-04-06


In [ ]:
remedios_demanda_regiao = pd.DataFrame()
remedios_demanda_estabelecimento = pd.DataFrame()

#**Janeiro**

In [ ]:
all_data_janeiro = pd.DataFrame()
all_data_janeiro = all_data
all_data_janeiro

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,87,2021-01-01
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,30,2021-01-01
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,4,2021-01-01
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,15,2021-01-01
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,496,2021-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
445622,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106400101100015,CLORPROMAZINA CLORIDRATO 5 MG/ML SOLUCAO INJET...,AMP,8410200104,DESBLOQUEADO,NAO,3.0,3,2021-01-31
445623,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106400101100139,HALOPERIDOL 5 MG/ML SOLUCAO INJETAVEL AMP. 1 ML,AMP,8410200104,DESBLOQUEADO,NAO,4.0,0,2021-01-31
445624,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106400101100147,"HALOPERIDOL DECANOATO 70,52 MG/ML (ESQUIVALENT...",AMP,8410200104,DESBLOQUEADO,NAO,15.0,17,2021-01-31
445625,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401005200226,DOXICICLINA CLORIDRATO 100 MG COMPRIMIDO,UN,8410200104,DESBLOQUEADO,NAO,1.0,20,2021-01-31


In [ ]:
all_data_janeiro.to_csv(r'all_data_janeiro', index = False)

In [ ]:
estoque_regiao_jan = all_data_janeiro.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['ESTOQUE'].sum()
estoque_regiao_jan

SUPERVISAO                              DESCRICAO ITEM                                                               data      
FUNDAÇÃO PARA O REMEDIO POPULAR - FURP  ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAPSULA OU DRAGEAS COM 250 MG     2021-01-01    33400
                                                                                                                     2021-01-02    33400
                                                                                                                     2021-01-03    33400
                                                                                                                     2021-01-04    33400
                                                                                                                     2021-01-05    33400
                                                                                                                                   ...  
SUPERVISÃO TÉCNICA TERMINAIS SPTRANS    DST/AIDS-P

In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.append(estoque_regiao_jan)

In [ ]:
estoque_estabelecimento_jan = all_data_janeiro.groupby(['ESTABELECIMENTO','DESCRICAO ITEM','data'])['ESTOQUE'].sum()
estoque_estabelecimento_jan

ESTABELECIMENTO       DESCRICAO ITEM                                     data      
AE ALTO DA BOA VISTA  ABAIXADOR, LINGUA, MADEIRA, 100 UNIDADES           2021-01-01    4
                                                                         2021-01-02    4
                                                                         2021-01-03    4
                                                                         2021-01-04    4
                                                                         2021-01-05    4
                                                                                      ..
URSI TUCURUVI         VASOPRESSINA 20 UI/ML SOLUCAO INJETAVEL AMP. 1 ML  2021-01-27    0
                                                                         2021-01-28    0
                                                                         2021-01-29    0
                                                                         2021-01-30    0
                          

In [ ]:
estoque_regiao_jan

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-03,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-04,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-05,33400
...,...,...,...,...
1236132,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-27,0
1236133,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-28,0
1236134,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-29,0
1236135,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-01-30,0


In [ ]:
estoque_regiao_jan.to_csv(r'estoque_regiao_jan.csv')

In [ ]:
#remedios_demanda_estabelecimento = remedios_demanda_estabelecimento.append(estoque_estabelecimento_jan)

#**Fevereiro**

In [ ]:
all_data_fevereiro = pd.DataFrame()
all_data_fevereiro = all_data
all_data_fevereiro

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,580,2021-02-01
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,490,2021-02-01
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,2,2021-02-01
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,13,2021-02-01
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,251,2021-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
446823,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,50,2021-02-28
446824,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,130,2021-02-28
446825,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,0,2021-02-28
446826,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,0,2021-02-28


In [ ]:
all_data_fevereiro.to_csv(r'all_data_fevereiro', index = False)

In [ ]:
estoque_regiao_fev = all_data_fevereiro.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['ESTOQUE'].sum()
estoque_regiao_fev

SUPERVISAO                              DESCRICAO ITEM                                                               data      
FUNDAÇÃO PARA O REMEDIO POPULAR - FURP  ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAPSULA OU DRAGEAS COM 250 MG     2021-02-01    33400
                                                                                                                     2021-02-02    33400
                                                                                                                     2021-02-03    33400
                                                                                                                     2021-02-04    33400
                                                                                                                     2021-02-05    33400
                                                                                                                                   ...  
SUPERVISÃO TÉCNICA TERMINAIS SPTRANS    DST/AIDS-P

In [ ]:
estoque_regiao_fev.to_csv(r'estoque_regiao_fev.csv')

In [ ]:
estoque_regiao_fev

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-03,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-04,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-02-05,33400
...,...,...,...,...
1117159,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-02-24,0
1117160,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-02-25,0
1117161,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-02-26,0
1117162,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-02-27,0


In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.append(estoque_regiao_fev)

#**Março**

In [ ]:
all_data_marco = pd.DataFrame()
all_data_marco = all_data
all_data_marco

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,580,2021-03-01
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,480,2021-03-01
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,2,2021-03-01
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,12,2021-03-01
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,136,2021-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
448291,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,50,2021-03-31
448292,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,130,2021-03-31
448293,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,0,2021-03-31
448294,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,0,2021-03-31


In [ ]:
all_data_marco.to_csv(r'all_data_marco', index = False)

In [ ]:
estoque_regiao_mar = all_data_marco.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['ESTOQUE'].sum()
estoque_regiao_mar

SUPERVISAO                              DESCRICAO ITEM                                                               data      
FUNDAÇÃO PARA O REMEDIO POPULAR - FURP  ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAPSULA OU DRAGEAS COM 250 MG     2021-03-01    33400
                                                                                                                     2021-03-02    33400
                                                                                                                     2021-03-03    33400
                                                                                                                     2021-03-04    33400
                                                                                                                     2021-03-05    33400
                                                                                                                                   ...  
SUPERVISÃO TÉCNICA TERMINAIS SPTRANS    DST/AIDS-P

In [ ]:
estoque_regiao_mar.to_csv(r'estoque_regiao_mar.csv')

In [ ]:
estoque_regiao_mar

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-03,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-04,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-03-05,33400
...,...,...,...,...
1238192,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-03-27,0
1238193,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-03-28,0
1238194,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-03-29,0
1238195,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-03-30,0


In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.append(estoque_regiao_mar)

#**Abril**

In [ ]:
all_data_abril = pd.DataFrame()
all_data_abril = all_data
all_data_abril

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,553,2021-04-01
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,470,2021-04-01
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,9,2021-04-01
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,12,2021-04-01
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,502,2021-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
449133,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,50,2021-04-30
449134,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,130,2021-04-30
449135,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,0,2021-04-30
449136,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,0,2021-04-30


In [ ]:
all_data_abril.to_csv(r'all_data_abril.csv')

In [ ]:
estoque_regiao_abr = all_data_abril.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['ESTOQUE'].sum()
estoque_regiao_abr

SUPERVISAO                              DESCRICAO ITEM                                                               data      
FUNDAÇÃO PARA O REMEDIO POPULAR - FURP  ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAPSULA OU DRAGEAS COM 250 MG     2021-04-01    33400
                                                                                                                     2021-04-02    33400
                                                                                                                     2021-04-03    33400
                                                                                                                     2021-04-04    33400
                                                                                                                     2021-04-05    33400
                                                                                                                                   ...  
SUPERVISÃO TÉCNICA TERMINAIS SPTRANS    DST/AIDS-P

In [ ]:
estoque_regiao_abr.to_csv(r'estoque_regiao_abr.csv')

In [ ]:
estoque_regiao_abr

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-03,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-04,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-04-05,33400
...,...,...,...,...
1201042,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-26,0
1201043,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-27,0
1201044,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-28,0
1201045,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-29,0


In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.append(estoque_regiao_abr)

In [ ]:
remedios_demanda_regiao

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-03,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-04,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-05,33400
...,...,...,...,...
1201042,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-26,0
1201043,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-27,0
1201044,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-28,0
1201045,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-04-29,0


#**Maio**

In [ ]:
all_data_maio = pd.DataFrame()
all_data_maio = all_data
all_data_maio

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,587,2021-05-21
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,470,2021-05-21
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,10,2021-05-21
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,13,2021-05-21
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,252,2021-05-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...
449191,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,50,2021-05-20
449192,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,130,2021-05-20
449193,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,0,2021-05-20
449194,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,0,2021-05-20


In [ ]:
all_data_maio.to_csv(r'all_data_maio', index = False)

In [ ]:
estoque_regiao_mai = all_data_maio.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['ESTOQUE'].sum()
estoque_regiao_mai

SUPERVISAO                              DESCRICAO ITEM                                                               data      
FUNDAÇÃO PARA O REMEDIO POPULAR - FURP  ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAPSULA OU DRAGEAS COM 250 MG     2021-05-01    33400
                                                                                                                     2021-05-02    33400
                                                                                                                     2021-05-03    33400
                                                                                                                     2021-05-04    33400
                                                                                                                     2021-05-05    33400
                                                                                                                                   ...  
SUPERVISÃO TÉCNICA TERMINAIS SPTRANS    DST/AIDS-P

In [ ]:
estoque_regiao_mai.to_csv(r'estoque_regiao_mai.csv')

In [ ]:
estoque_regiao_mai

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-03,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-04,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-05-05,33400
...,...,...,...,...
1242245,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-05-27,0
1242246,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-05-28,0
1242247,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-05-29,0
1242248,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-05-30,0


In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.append(estoque_regiao_mai)

#**Junho**

In [ ]:
all_data_junho = pd.DataFrame()
all_data_junho = all_data
all_data_junho

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,587,2021-06-01
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,470,2021-06-01
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,10,2021-06-01
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,13,2021-06-01
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,202,2021-06-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
453023,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,40,2021-06-30
453024,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,130,2021-06-30
453025,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,4,2021-06-30
453026,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,0,2021-06-30


In [ ]:
all_data_junho.to_csv(r'all_data_junho', index = False)

In [ ]:
estoque_regiao_jun = all_data_junho.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['ESTOQUE'].sum()
estoque_regiao_jun

SUPERVISAO                              DESCRICAO ITEM                                                               data      
FUNDAÇÃO PARA O REMEDIO POPULAR - FURP  ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAPSULA OU DRAGEAS COM 250 MG     2021-06-01    33400
                                                                                                                     2021-06-02    33400
                                                                                                                     2021-06-03    33400
                                                                                                                     2021-06-04    33400
                                                                                                                     2021-06-05    33400
                                                                                                                                   ...  
SUPERVISÃO TÉCNICA TERMINAIS SPTRANS    DST/AIDS-P

In [ ]:
estoque_regiao_jun.to_csv(r'estoque_regiao_jun.csv')

In [ ]:
estoque_regiao_jun

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-03,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-04,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-06-05,33400
...,...,...,...,...
1206303,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-06-26,0
1206304,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-06-27,0
1206305,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-06-28,0
1206306,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-06-29,0


In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.append(estoque_regiao_jun)

#**Julho**

In [ ]:
all_data_julho = pd.DataFrame()
all_data_julho = all_data
all_data_julho

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,520,2021-07-01
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,430,2021-07-01
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,7,2021-07-01
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,8,2021-07-01
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,62,2021-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
459124,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,40,2021-07-31
459125,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,100,2021-07-31
459126,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,4,2021-07-31
459127,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,0,2021-07-31


In [ ]:
all_data_julho.to_csv(r'all_data_julho', index = False)

In [ ]:
estoque_regiao_jul = all_data_julho.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['ESTOQUE'].sum()
estoque_regiao_jul

SUPERVISAO                              DESCRICAO ITEM                                                               data      
FUNDAÇÃO PARA O REMEDIO POPULAR - FURP  ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAPSULA OU DRAGEAS COM 250 MG     2021-07-01    33400
                                                                                                                     2021-07-02    33400
                                                                                                                     2021-07-03    33400
                                                                                                                     2021-07-04    33400
                                                                                                                     2021-07-05    33400
                                                                                                                                   ...  
SUPERVISÃO TÉCNICA TERMINAIS SPTRANS    DST/AIDS-P

In [ ]:
estoque_regiao_jul.to_csv(r'estoque_regiao_jul.csv')

In [ ]:
estoque_regiao_jul

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-03,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-04,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-07-05,33400
...,...,...,...,...
1270230,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-07-27,0
1270231,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-07-28,0
1270232,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-07-29,0
1270233,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-07-30,0


In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.append(estoque_regiao_jul)

#**Agosto**

In [ ]:
all_data_agosto = pd.DataFrame()
all_data_agosto = all_data
all_data_agosto

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,467,2021-08-01
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,430,2021-08-01
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,107,2021-08-01
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,5,2021-08-01
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,962,2021-08-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
467728,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,40,2021-08-31
467729,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,100,2021-08-31
467730,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,4,2021-08-31
467731,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,1,2021-08-31


In [ ]:
all_data_agosto.to_csv(r'all_data_agosto', index = False)

In [ ]:
estoque_regiao_ago = all_data_agosto.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['ESTOQUE'].sum()
estoque_regiao_ago

SUPERVISAO                              DESCRICAO ITEM                                                               data      
FUNDAÇÃO PARA O REMEDIO POPULAR - FURP  ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAPSULA OU DRAGEAS COM 250 MG     2021-08-01    33400
                                                                                                                     2021-08-02    33400
                                                                                                                     2021-08-04    33400
                                                                                                                     2021-08-05    33400
                                                                                                                     2021-08-06    33400
                                                                                                                                   ...  
SUPERVISÃO TÉCNICA TERMINAIS SPTRANS    DST/AIDS-P

In [ ]:
estoque_regiao_ago.to_csv(r'estoque_regiao_ago.csv')

In [ ]:
estoque_regiao_ago

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-04,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-05,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-08-06,33400
...,...,...,...,...
1246534,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-08-27,0
1246535,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-08-28,0
1246536,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-08-29,0
1246537,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-08-30,0


In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.append(estoque_regiao_ago)

In [ ]:
estoque_estabelecimento_ago = all_data_agosto.groupby(['ESTABELECIMENTO','DESCRICAO ITEM','data'])['ESTOQUE'].sum()
estoque_estabelecimento_ago

ESTABELECIMENTO       DESCRICAO ITEM                                     data      
AE ALTO DA BOA VISTA  ABAIXADOR, LINGUA, MADEIRA, 100 UNIDADES           2021-08-01    4
                                                                         2021-08-02    4
                                                                         2021-08-04    4
                                                                         2021-08-05    4
                                                                         2021-08-06    4
                                                                                      ..
URSI TUCURUVI         VASOPRESSINA 20 UI/ML SOLUCAO INJETAVEL AMP. 1 ML  2021-08-27    2
                                                                         2021-08-28    2
                                                                         2021-08-29    2
                                                                         2021-08-30    2
                          

In [ ]:
remedios_demanda_estabelecimento = remedios_demanda_estabelecimento.append(estoque_estabelecimento_ago)

#**Setembro**

In [ ]:
all_data_setembro = pd.DataFrame()
all_data_setembro = all_data
all_data_setembro

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,581,2021-09-22
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,540,2021-09-22
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,103,2021-09-22
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,5,2021-09-22
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,459,2021-09-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
469587,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,40,2021-09-21
469588,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,100,2021-09-21
469589,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,4,2021-09-21
469590,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,1,2021-09-21


In [ ]:
all_data_setembro.to_csv(r'all_data_setembro', index = False)

In [ ]:
estoque_regiao_set = all_data_setembro.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['ESTOQUE'].sum()
estoque_regiao_set

SUPERVISAO                              DESCRICAO ITEM                                                               data      
FUNDAÇÃO PARA O REMEDIO POPULAR - FURP  ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAPSULA OU DRAGEAS COM 250 MG     2021-09-01    33400
                                                                                                                     2021-09-02    33400
                                                                                                                     2021-09-03    33400
                                                                                                                     2021-09-04    33400
                                                                                                                     2021-09-05    33400
                                                                                                                                   ...  
SUPERVISÃO TÉCNICA TERMINAIS SPTRANS    DST/AIDS-P

In [ ]:
estoque_regiao_set.to_csv(r'estoque_regiao_set.csv')

In [ ]:
estoque_regiao_set

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-03,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-04,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-09-05,33400
...,...,...,...,...
1254239,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-09-26,0
1254240,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-09-27,0
1254241,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-09-28,0
1254242,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-09-29,0


In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.append(estoque_regiao_set)

In [ ]:
estoque_estabelecimento_set = all_data_setembro.groupby(['ESTABELECIMENTO','DESCRICAO ITEM','data'])['ESTOQUE'].sum()
estoque_estabelecimento_set

ESTABELECIMENTO       DESCRICAO ITEM                                     data      
AE ALTO DA BOA VISTA  ABAIXADOR, LINGUA, MADEIRA, 100 UNIDADES           2021-09-01    4
                                                                         2021-09-02    4
                                                                         2021-09-03    4
                                                                         2021-09-04    4
                                                                         2021-09-05    4
                                                                                      ..
URSI TUCURUVI         VASOPRESSINA 20 UI/ML SOLUCAO INJETAVEL AMP. 1 ML  2021-09-26    2
                                                                         2021-09-27    2
                                                                         2021-09-28    2
                                                                         2021-09-29    2
                          

In [ ]:
remedios_demanda_estabelecimento = remedios_demanda_estabelecimento.append(estoque_estabelecimento_set)

#**Outubro**

In [ ]:
all_data_outubro = pd.DataFrame()
all_data_outubro = all_data
all_data_outubro

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,581,2021-10-01
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,530,2021-10-01
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,102,2021-10-01
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,5,2021-10-01
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,179,2021-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
486401,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,40,2021-10-31
486402,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,100,2021-10-31
486403,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,4,2021-10-31
486404,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,1,2021-10-31


In [ ]:
all_data_outubro.to_csv(r'all_data_outubro', index = False)

In [ ]:
estoque_regiao_out = all_data_outubro.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['ESTOQUE'].sum()
estoque_regiao_out

SUPERVISAO                              DESCRICAO ITEM                                                               data      
FUNDAÇÃO PARA O REMEDIO POPULAR - FURP  ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAPSULA OU DRAGEAS COM 250 MG     2021-10-01    33400
                                                                                                                     2021-10-02    33400
                                                                                                                     2021-10-03    33400
                                                                                                                     2021-10-04    33400
                                                                                                                     2021-10-05    33400
                                                                                                                                   ...  
SUPERVISÃO TÉCNICA TERMINAIS SPTRANS    DST/AIDS-P

In [ ]:
estoque_regiao_out.to_csv(r'estoque_regiao_out.csv')

In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.append(estoque_regiao_out)

In [ ]:
estoque_regiao_out

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-03,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-04,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-10-05,33400
...,...,...,...,...
1299871,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-10-27,0
1299872,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-10-28,0
1299873,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-10-29,0
1299874,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-10-30,0


In [ ]:
remedios_demanda_estabelecimento = remedios_demanda_estabelecimento.append(estoque_estabelecimento_out)

#**Novembro**

In [ ]:
all_data_novembro = pd.DataFrame()
all_data_novembro = all_data
all_data_novembro

,SUPERVISAO,CNES ESTABELECIMENTO,SUPRI ESTABELECIMENTO,ESTABELECIMENTO,CODIGO SUPRI ITEM,DESCRICAO ITEM,TIPO UNIDADE MEDIDA,CODIGO ALMOXARIFADO ITEM,STATUS,BLOQUEIO STS,CMM,ESTOQUE,data
0,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100011,PARACETAMOL 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,100.0,579,2021-11-01
1,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100038,ACIDO ACETILSALICILICO 100 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,50.0,496,2021-11-01
2,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100054,DIPIRONA SODICA 500 MG/ML SOLUCAO ORAL GOTAS F...,FR,8410200066,DESBLOQUEADO,NAO,10.0,100,2021-11-01
3,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100062,DIPIRONA SODICA 500 MG/ML SOLUCAO INJETAVEL AM...,AMP,8410200066,DESBLOQUEADO,NAO,5.0,5,2021-11-01
4,SUPERVISÃO TÉCNICA DE SAÚDE SÉ,2786532.0,8.428300e+09,CAPS AD III CENTRO,1106400100100097,DIPIRONA SODICA 500 MG COMPRIMIDO,UN,8410200066,DESBLOQUEADO,NAO,300.0,410,2021-11-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
486618,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200505,OSELTAMIVIR 45 MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,40,2021-11-17
486619,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401006200513,OSELTAMIVIR 3O MG CAPSULA,UN,8410200104,DESBLOQUEADO,NAO,50.0,100,2021-11-17
486620,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401306500300,"NALOXONA CLORIDRATO 0,4 MG/ML SOLUCAO INJETAVE...",AMP,8410200104,DESBLOQUEADO,NAO,4.0,4,2021-11-17
486621,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,2091348.0,8.427300e+09,CSE BUTANTA SAMUEL B PESSOA,1106401807000504,CLORANFENICOL 5 MG/G + RETINOL ACETATO 10.000U...,BNG,8410200104,DESBLOQUEADO,NAO,1.0,1,2021-11-17


In [ ]:
all_data_novembro.to_csv(r'all_data_novembro', index = False)

In [ ]:
estoque_regiao_nov = all_data_novembro.groupby(['SUPERVISAO','DESCRICAO ITEM','data'])['ESTOQUE'].sum()
estoque_regiao_nov

SUPERVISAO                              DESCRICAO ITEM                                                               data      
FUNDAÇÃO PARA O REMEDIO POPULAR - FURP  ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAPSULA OU DRAGEAS COM 250 MG     2021-11-01    33400
                                                                                                                     2021-11-02    33400
                                                                                                                     2021-11-03    33400
                                                                                                                     2021-11-04    33400
                                                                                                                     2021-11-05    33400
                                                                                                                                   ...  
SUPERVISÃO TÉCNICA TERMINAIS SPTRANS    DST/AIDS-P

In [ ]:
estoque_regiao_nov.to_csv(r'estoque_regiao_nov.csv')

In [ ]:
estoque_regiao_nov

,SUPERVISAO,DESCRICAO ITEM,data,ESTOQUE
0,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-01,33400
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-02,33400
2,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-03,33400
3,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-04,33400
4,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-11-05,33400
...,...,...,...,...
743612,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-13,0
743613,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-14,0
743614,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-15,0
743615,SUPERVISÃO TÉCNICA TERMINAIS SPTRANS,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-16,0


In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.append(estoque_regiao_nov)

In [ ]:
remedios_demanda_estabelecimento = remedios_demanda_estabelecimento.append(estoque_estabelecimento_nov)

#**Estoque região - único arquivo**

In [ ]:
remedios_demanda_regiao = pd.read_csv("remedios_estoque_fato_regiao.csv")

In [ ]:
remedios_demanda_regiao

,id_supervisao,id_descricao_item,data,estoque
0,1,61,2021-01-01,33400
1,1,61,2021-01-02,33400
2,1,61,2021-01-03,33400
3,1,61,2021-01-04,33400
4,1,61,2021-01-05,33400
...,...,...,...,...
13055609,32,1298,2021-11-13,0
13055610,32,1298,2021-11-14,0
13055611,32,1298,2021-11-15,0
13055612,32,1298,2021-11-16,0


#**Salvando**

In [ ]:
remedios_demanda_regiao

,id_supervisao,id_descricao_item,data,estoque
0,1,61,2021-01-01,33400
1,1,61,2021-01-02,33400
2,1,61,2021-01-03,33400
3,1,61,2021-01-04,33400
4,1,61,2021-01-05,33400
...,...,...,...,...
13055609,32,1298,2021-11-13,0
13055610,32,1298,2021-11-14,0
13055611,32,1298,2021-11-15,0
13055612,32,1298,2021-11-16,0


In [ ]:
remedios_demanda_regiao = remedios_demanda_regiao.rename(columns={'SUPERVISAO':'supervisao','DESCRICAO ITEM':'descricao_item','ESTOQUE':'estoque'})

In [ ]:
remedios_demanda_regiao.to_csv(r'/remedios_estoque_regiao_fato.csv', index = False)

In [ ]:
remedios_demanda_regiao

,id_supervisao,id_descricao_item,data,estoque
0,1,61,2021-01-01,33400
1,1,61,2021-01-02,33400
2,1,61,2021-01-03,33400
3,1,61,2021-01-04,33400
4,1,61,2021-01-05,33400
...,...,...,...,...
13055609,32,1298,2021-11-13,0
13055610,32,1298,2021-11-14,0
13055611,32,1298,2021-11-15,0
13055612,32,1298,2021-11-16,0


#**remedios_estoque_regiao_dim_supervisao**

In [ ]:
estoque_regiao_dim_supervisao = pd.DataFrame()
estoque_regiao_dim_supervisao['supervisao'] = remedios_demanda_regiao['supervisao'].drop_duplicates()
estoque_regiao_dim_supervisao = estoque_regiao_dim_supervisao.sort_values(by=['supervisao'], ascending=True)
estoque_regiao_dim_supervisao.index = np.arange(1, len(estoque_regiao_dim_supervisao) + 1)
estoque_regiao_dim_supervisao['id_supervisao'] = estoque_regiao_dim_supervisao.index

In [ ]:
estoque_regiao_dim_supervisao

,supervisao,id_supervisao
1,FUNDAÇÃO PARA O REMEDIO POPULAR - FURP,1
2,SECRETARIA MUNICIPAL DA SAUDE,2
3,SUPERVISAO TECNICA DE SAUDE ERMELINO MATARAZZO,3
4,SUPERVISAO TECNICA DE SAUDE HOSPITALAR,4
5,SUPERVISAO TECNICA DE SAUDE PERUS,5
6,SUPERVISAO TECNICA DE SAUDE SANTA CECILIA,6
7,SUPERVISAO TECNICA DE SAUDE SAO MIGUEL,7
8,SUPERVISÃO TÉCNICA DE SAÚDE ARICANDUVA/FORMOSA...,8
9,SUPERVISÃO TÉCNICA DE SAÚDE BUTANTÃ,9
10,SUPERVISÃO TÉCNICA DE SAÚDE CAMPO LIMPO,10


In [ ]:
estoque_regiao_dim_supervisao.to_csv(r'remedios_estoque_regiao_dim_supervisao.csv', index = False)

In [ ]:
def ConverteSupervisaoNumID(id_supervisao):
    dicionario_supervisao = dict(sorted(estoque_regiao_dim_supervisao.values.tolist())) 
    return dicionario_supervisao[id_supervisao]

In [ ]:
n = 0
lista = []
while n < len(remedios_demanda_regiao): 
  id_supervisao = remedios_demanda_regiao['supervisao'][n]
  supervisao = ConverteSupervisaoNumID(id_supervisao)
  lista.insert(n, supervisao)
  n += 1

In [ ]:
remedios_demanda_regiao['supervisao']= lista
remedios_demanda_regiao = remedios_demanda_regiao.rename(columns = {'supervisao': 'id_supervisao'}, inplace = False)

In [ ]:
remedios_demanda_regiao

,id_supervisao,descricao_item,data,estoque
0,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-01,33400
1,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-02,33400
2,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-03,33400
3,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-04,33400
4,1,"ACIDO VALPROICO EM COMPRIMIDOS REVESTIDOS, CAP...",2021-01-05,33400
...,...,...,...,...
13055609,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-13,0
13055610,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-14,0
13055611,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-15,0
13055612,32,DST/AIDS-PRESERVATIVO MASCULINO C/ LUBRIFICANT...,2021-11-16,0


#**remedios_estoque_regiao_dim_descricao_item**

In [ ]:
estoque_regiao_dim_descricao_item = pd.DataFrame()
estoque_regiao_dim_descricao_item['descricao_item'] = remedios_demanda_regiao['descricao_item'].drop_duplicates()
estoque_regiao_dim_descricao_item = estoque_regiao_dim_descricao_item.sort_values(by=['descricao_item'], ascending=True)
estoque_regiao_dim_descricao_item.index = np.arange(1, len(estoque_regiao_dim_descricao_item) + 1)
estoque_regiao_dim_descricao_item['id_descricao_item'] = estoque_regiao_dim_descricao_item.index

In [ ]:
estoque_regiao_dim_descricao_item

,descricao_item,id_descricao_item
1,-- NAO UTILIZAR -- ALENDRONATO DE SODIO EM COM...,1
2,-- NAO UTILIZAR -- ASSOCIACAO DE LEVODOPA 200 ...,2
3,-- NAO UTILIZAR -- LEVODOPA 100 MG + CLORIDRAT...,3
4,-- NAO UTILIZAR -- LEVODOPA 200 MG + CLORIDRAT...,4
5,-- NAO UTILIZAR -- SINVASTATINA EM COMPRIMIDOS...,5
...,...,...
3279,WATER TRAP 1 ADULTO/PEDIATRICO,3279
3280,"XILOL, P.A.",3280
3281,ZANAMIVIR 5 MG PO PARA SOLUCAO INALANTE,3281
3282,ZIDOVUDINA (AZT) 10 MG/ML SOLUCAO ORAL FRASCO ...,3282


In [ ]:
estoque_regiao_dim_descricao_item.to_csv(r'remedios_estoque_regiao_dim_descricao_item.csv', index = False)

In [ ]:
def Converte_descricao_item_NumID(id_descricao_item):
    dicionario_descricao_item = dict(sorted(estoque_regiao_dim_descricao_item.values.tolist())) 
    return dicionario_descricao_item[id_descricao_item]

In [ ]:
n = 0
lista = []
while n < (len(remedios_demanda_regiao)/10): 
  id_descricao_item = remedios_demanda_regiao['descricao_item'][n]
  descricao_item = Converte_descricao_item_NumID(id_descricao_item)
  lista.insert(n, descricao_item)
  n += 1

In [ ]:
n = (int(len(remedios_demanda_regiao))/10)*1
lista = []
while n < (len(remedios_demanda_regiao)/10)*2: 
  id_descricao_item = remedios_demanda_regiao['descricao_item'][n]
  descricao_item = Converte_descricao_item_NumID(id_descricao_item)
  lista.insert(n, descricao_item)
  n += 1

KeyError: ignored

In [ ]:
n = (len(remedios_demanda_regiao)/10)*2
lista = []
while n < (len(remedios_demanda_regiao)/10)*3: 
  id_descricao_item = remedios_demanda_regiao['descricao_item'][n]
  descricao_item = Converte_descricao_item_NumID(id_descricao_item)
  lista.insert(n, descricao_item)
  n += 1

In [ ]:
remedios_demanda_regiao['descricao_item']= lista
remedios_demanda_regiao = remedios_demanda_regiao.rename(columns = {'descricao_item': 'id_descricao_item'}, inplace = False)

In [ ]:
remedios_demanda_regiao.to_csv(r'remedios_estoque_regiao_fato_incomplete_1.csv', index = False)

#Provas reais

In [ ]:
data_n = extraidos[0].split('/')
data_n = data_n[9].split('_')
data_n = data_n[3].split('.')
data_n = int(data_n[0])
data_n

20210101

In [ ]:
data_n_ano = data_n // 10000
data_n_ano

2021

In [ ]:
data_n_mes = (data_n % 10000)//100
data_n_mes 

9

In [ ]:
data_n_dia = data_n % 10000
if data_n_dia > 999:
  data_n_dia = data_n_dia%100
else:
  data_n_dia = data_n_dia%100
data_n_dia

23

In [ ]:
date(year=data_n_ano, month=data_n_mes, day=data_n_dia)

datetime.date(2021, 9, 23)

In [ ]:
for f in extraidos:
  print(f)

/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/gss_posicao_estoque_20210923.csv
/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/gss_posicao_estoque_20210924.csv
/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/gss_posicao_estoque_20210925.csv
/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/gss_posicao_estoque_20210926.csv
/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/gss_posicao_estoque_20210927.csv
/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/gss_posicao_estoque_20210928.csv
/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/gss_posicao_estoque_20210929.csv
/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/gss_posicao_estoque_20210930.csv
/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/gss_posicao_estoque_20211001.csv
/content/drive/My Drive/Prefeitura/Dados/situa

In [ ]:
for f in extraidos:
  data_n = extraidos[f].split('/')
  data_n = data_n[8].split('_')
  data_n = data_n[3].split('.')
  data_n = data_n[0]
  data_n = str(data_n)
  data_n = int(data_n)
  print(data_n)

TypeError: ignored

#lab de operações!



In [ ]:
#Se houvesse vários arquivos esse codigo era uma boa!
#for n in arquivos:
#    for file in arquivos[0].namelist():
#        arquivos.extract(file, '/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/extraidos/')

In [ ]:
with zipfile.ZipFile('/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/Novembro/gss_posicao_estoque_20211102.zip') as zip:
    lista = z.namelist()

In [ ]:
lista

['gss_posicao_estoque_20211101.csv']

In [ ]:
with zipfile.ZipFile('/content/drive/My Drive/Prefeitura/Dados/situacao_medicamentos/Novembro/gss_posicao_estoque_20211102.zip') as zip:
  with z.open('gss_posicao_estoque_20211101.csv') as f:
    df = pd.read_csv(f, sep = ';', header = 0, encoding='ISO-8859-1', nrows=0 )

ValueError: ignored

In [ ]:
print(df)

                                                    0
0   /content/drive/My Drive/Prefeitura/Dados/situa...
1   /content/drive/My Drive/Prefeitura/Dados/situa...
2   /content/drive/My Drive/Prefeitura/Dados/situa...
3   /content/drive/My Drive/Prefeitura/Dados/situa...
4   /content/drive/My Drive/Prefeitura/Dados/situa...
5   /content/drive/My Drive/Prefeitura/Dados/situa...
6   /content/drive/My Drive/Prefeitura/Dados/situa...
7   /content/drive/My Drive/Prefeitura/Dados/situa...
8   /content/drive/My Drive/Prefeitura/Dados/situa...
9   /content/drive/My Drive/Prefeitura/Dados/situa...
10  /content/drive/My Drive/Prefeitura/Dados/situa...
11  /content/drive/My Drive/Prefeitura/Dados/situa...
12  /content/drive/My Drive/Prefeitura/Dados/situa...
13  /content/drive/My Drive/Prefeitura/Dados/situa...
14  /content/drive/My Drive/Prefeitura/Dados/situa...
15  /content/drive/My Drive/Prefeitura/Dados/situa...
16  /content/drive/My Drive/Prefeitura/Dados/situa...


In [ ]:
data_frame = pd.DataFrame()
for n in arquivos:
    with zipfile.ZipFile(n) as z:
        with z.open(n) as f:
            dados = pd.read_csv(f, sep = ';', header = 0, encoding='ISO-8859-1',nrows=0)
            all_data = pd.concat(dados, axis = 1)